In [16]:
import pandas as pd

In [17]:
# Input file after dropping rows with missing Total Spent
CSV_IN = "../../output/1_handle_missing_data/price_per_unit_reconstructed.csv"
CSV_OUT = "../../output/1_handle_missing_data/item_imputed.csv"

# Define columns name
TOTAL_SPENT = "Total Spent"
PRICE_PER_UNIT = "Price Per Unit"
QUANTITY = "Quantity"
CATEGORY = "Category"
PAYMENT_METHOD = "Payment Method"
LOCATION = "Location"
ITEM = "Item"
TRANSACTION_ID = "Transaction ID"

# Define error
COERCE_ERRORS = "coerce"

In [18]:
# Load dataset
df = pd.read_csv(CSV_IN)

### Quantify missing Item

Determine the scale of missing entries in `Item` after STEP 2 completion. Since Item cannot be deterministically reconstructed (no formula exists), statistical imputation using Category information will be required.


In [ ]:
# Count missing Item values
missingItem = df[ITEM].isna()
print(f'Missing Item rows: {missingItem.sum()} of {len(df)} ({missingItem.mean():.2%})')

# Verify that numeric columns are complete
print(f'\nVerification of previous steps:')
print(f'  Total Spent missing: {df[TOTAL_SPENT].isna().sum()}')
print(f'  Quantity missing: {df[QUANTITY].isna().sum()}')
print(f'  Price Per Unit missing: {df[PRICE_PER_UNIT].isna().sum()}')


### Missingness mechanism

Quantifying how often `Item` is missing within each `Category`, payment method, and location to understand the systematic pattern. The variation across categories confirms MAR classification.


In [ ]:
# Analyze missingness patterns across categories
summary = df.assign(missingItem=missingItem).groupby(CATEGORY)['missingItem'].mean().sort_values(ascending=False) * 100
print('Share of Item missing by Category:')
print(summary.round(2).astype(str) + '%')

# Analyze missingness patterns across payment methods
paymentShare = df.assign(missingItem=missingItem).groupby(PAYMENT_METHOD)['missingItem'].mean().sort_values(ascending=False) * 100
print('\nShare of Item missing by Payment Method:')
print(paymentShare.round(2).astype(str) + '%')

# Analyze missingness patterns across locations
locationShare = df.assign(missingItem=missingItem).groupby(LOCATION)['missingItem'].mean().sort_values(ascending=False) * 100
print('\nShare of Item missing by Location:')
print(locationShare.round(2).astype(str) + '%')


### Category coverage analysis

Verify that all missing Item rows have valid Category information, which is essential for category-based imputation.


In [ ]:
# Check if all missing Item rows have valid Category
itemAndCategoryMissing = (missingItem & df[CATEGORY].isna()).sum()

print(f'Rows with both Item and Category missing: {itemAndCategoryMissing}')
print(f'Rows with Item missing but Category present: {missingItem.sum() - itemAndCategoryMissing}')
print(f'Category coverage for missing Items: {((missingItem.sum() - itemAndCategoryMissing) / missingItem.sum() * 100):.1f}%')


### Item distribution analysis

Examine the distribution of Items within each Category to understand what values are available for imputation. The mode (most frequent item) per category will be used.


In [21]:
# Analyze Item distribution within each Category
# Calculate mode (most frequent item) for each category
# We'll use this for imputation
for category in df[CATEGORY].unique():
    # Filter data to current category and non-missing Items
    categoryData = df[(df[CATEGORY] == category) & (df[ITEM].notna())]
    
    if len(categoryData) > 0:
        # .mode() returns the most frequent value(s)
        # [0] takes the first mode if there are multiple
        modeItem = categoryData[ITEM].mode()[0]
        # Count how many times this item appears
        modeCount = (categoryData[ITEM] == modeItem).sum()
        # Calculate percentage
        modePct = (modeCount / len(categoryData)) * 100
        # Count how many Items will be imputed for this category
        toImpute = ((df[CATEGORY] == category) & missingItem).sum()
        
        print(f'{category:40s}: {modeItem:20s} (appears {modeCount:4d} times, {modePct:5.1f}%) -> will impute {toImpute} rows')

# Show unique item counts per category
print('\nItem variety per Category:')
itemVariety = df[df[ITEM].notna()].groupby(CATEGORY)[ITEM].nunique().sort_values(ascending=False)
for category, count in itemVariety.items():
    print(f'{category:40s}: {count:3d} unique items')


Food                                    : Item_14_FOOD         (appears  106 times,   7.4%) -> will impute 81 rows
Furniture                               : Item_25_FUR          (appears  113 times,   7.7%) -> will impute 65 rows
Computers and electric accessories      : Item_19_CEA          (appears  106 times,   7.6%) -> will impute 80 rows
Milk Products                           : Item_16_MILK         (appears  109 times,   7.6%) -> will impute 88 rows
Electric household essentials           : Item_8_EHE           (appears  105 times,   7.3%) -> will impute 79 rows
Beverages                               : Item_2_BEV           (appears  126 times,   8.8%) -> will impute 69 rows
Butchers                                : Item_20_BUT          (appears  107 times,   7.5%) -> will impute 75 rows
Patisserie                              : Item_12_PAT          (appears  100 times,   7.3%) -> will impute 72 rows

Item variety per Category:
Beverages                               :  25 unique

### Missing data classification

**Classification: MAR (Missing At Random)**

**Rationale:**
- ALL 609 missing Items have valid Category information (100% coverage)
- The missingness depends on Category (an observable variable)
- Not MCAR because missing rates are not uniform across categories
- Not MNAR because missingness is explained by Category, not by the item values themselves



### Handling strategy: Mode imputation by Category

**Reason for choose mode imputation (not mean/median or deletion):**

1. **Strong predictor available:** Category provides strong signal for Item prediction (100% coverage)
2. **Preserves distribution:** Mode imputation maintains the frequency distribution within each category
3. **Conservative approach:** Uses actual existing item codes, doesn't create synthetic values
4. **Appropriate for categorical df:** Mode is the standard measure for categorical variables
5. **Small data loss if deleted:** Dropping 609 rows (5.09%) would lose valuable data unnecessarily

**Why mode (not other methods):**
- **Mean/Median:** Not applicable to categorical (text) data
- **Deletion:** Would lose 5.09% of data when imputation is feasible
- **Random imputation:** Less stable than mode, introduces unnecessary variance
- **Create new category:** Would violate existing item naming convention
- **Predictive model:** More complex, may overfit with limited missing df




In [22]:

print(df[missingItem][[TRANSACTION_ID, CATEGORY, ITEM, PRICE_PER_UNIT, QUANTITY, TOTAL_SPENT]].head(10))


    Transaction ID                            Category Item  Price Per Unit  \
12     TXN_1007496                            Butchers  NaN            15.5   
50     TXN_1032287                                Food  NaN            21.5   
68     TXN_1044590       Electric household essentials  NaN            14.0   
70     TXN_1046262                       Milk Products  NaN            14.0   
71     TXN_1046367  Computers and electric accessories  NaN            18.5   
76     TXN_1051223                          Patisserie  NaN             5.0   
87     TXN_1058643                                Food  NaN             9.5   
104    TXN_1071762                           Beverages  NaN             9.5   
134    TXN_1095879                           Beverages  NaN             6.5   
136    TXN_1096977                                Food  NaN            23.0   

     Quantity  Total Spent  
12       10.0        155.0  
50        2.0         43.0  
68        4.0         56.0  
70        5.0 

In [23]:
# Count missing values before imputation
itemMissingBefore = missingItem.sum()
print(f'Item missing before imputation: {itemMissingBefore}')

# Perform mode imputation by Category
# Create a dictionary to track imputation details for reporting
imputationDetails = []

# Iterate through each unique category
# df[CATEGORY].unique() returns all unique category values
for category in df[CATEGORY].unique():
    # Create filter for rows in this category with missing Item
    # (df[CATEGORY] == category) filters to current category
    # & missing_item filters to rows with missing Item
    # Both conditions must be True
    categoryMask = (df[CATEGORY] == category) & missingItem
    missingInCategory = categoryMask.sum()
    
    if missingInCategory > 0:
        # Get non-missing Items in this category to find the mode
        # Filter to current category AND non-missing Items
        categoryItems = df[(df[CATEGORY] == category) & (df[ITEM].notna())][ITEM]
        
        if len(categoryItems) > 0:
            # .mode() returns the most frequent value(s) as a Series
            # [0] takes the first mode if there are multiple modes
            modeItem = categoryItems.mode()[0]
            
            # Perform imputation: assign mode_item to all missing Items in this category
            # .loc[filter, column] allows us to update specific rows and columns
            df.loc[categoryMask, ITEM] = modeItem
            
            # Track details for reporting
            imputationDetails.append({
                'Category': category,
                'Missing Count': missingInCategory,
                'Imputed With': modeItem
            })

# Count missing values after imputation
# df[ITEM].isna().sum() recounts missing values after imputation
itemMissingAfter = df[ITEM].isna().sum()
# Calculate how many values were successfully imputed
valuesImputed = itemMissingBefore - itemMissingAfter

print(f'\nItem missing after imputation: {itemMissingAfter}')
print(f'Values successfully imputed: {valuesImputed}')


# Display imputation details
print('Imputation Details by Category:')

# Create DataFrame from imputation details for formatting
imputation_df = pd.DataFrame(imputationDetails)
print(imputation_df.to_string(index=False))


Item missing before imputation: 609

Item missing after imputation: 0
Values successfully imputed: 609
Imputation Details by Category:
                          Category  Missing Count Imputed With
                              Food             81 Item_14_FOOD
                         Furniture             65  Item_25_FUR
Computers and electric accessories             80  Item_19_CEA
                     Milk Products             88 Item_16_MILK
     Electric household essentials             79   Item_8_EHE
                         Beverages             69   Item_2_BEV
                          Butchers             75  Item_20_BUT
                        Patisserie             72  Item_12_PAT


### Validation: Verify imputation correctness

Verify that the imputed Item values maintain category consistency and that all items follow the correct naming convention


In [25]:
# Verify Item is now complete
print('Missing value check after imputation:')
print(f'Item missing: {df[ITEM].isna().sum()}')
print(f'Price Per Unit missing: {df[PRICE_PER_UNIT].isna().sum()}')
print(f'Quantity missing: {df[QUANTITY].isna().sum()}')
print(f'Total Spent missing: {df[TOTAL_SPENT].isna().sum()}')


Missing value check after imputation:
Item missing: 0
Price Per Unit missing: 0
Quantity missing: 0
Total Spent missing: 0


### Impact on remaining missing values

Analyze the current state of missing data after Item imputation. Only Discount Applied should have missing values remaining


In [28]:
print('Current missing value status across all columns:')

# Check all columns for missing values
missingSummary = df.isnull().sum()
# Filter to show only columns with missing values
missingCols = missingSummary[missingSummary > 0]

print(missingCols)


Current missing value status across all columns:
Discount Applied    3988
dtype: int64


### Persist results

Save the dataset with Item imputed. This becomes the input for STEP 4 (Discount Applied handling)


In [ ]:
# Save dataset with Item imputed to CSV
df.to_csv(CSV_OUT, index=False)

### Summary

**Item Handling**

**Classification:** MAR (Missing At Random)
- Missingness depends on Category (observable variable)
- Missing rates vary by category (8.23%-10.41%)
- ALL missing Items have valid Category information (100%)

**Method:** Mode imputation by Category
- Imputed 609 values (100% of missing items)
- Used most frequent item within each category
- Conservative approach using existing item codes

**Justification:**
- Category provides strong predictive signal (100% coverage)
- Mode preserves distribution within categories
- Appropriate method for categorical data
- Maintains category consistency
